# OPT Embeddings with HuggingFace transformers
I think this will be similar to our process with BERT.

In [14]:
# Documentation says to use the GPT2 Tokenizer here, not sure why.
from transformers import GPT2Tokenizer, OPTModel
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-1.3b")
model = OPTModel.from_pretrained("facebook/opt-1.3b")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/685 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/653 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.45G [00:00<?, ?B/s]

### Vocabulary time!
I'm going to try and skip a lot of old code by asking the tokenizer to give me PyTorch tensors from the get-go.

In [3]:
def squeeze(word, tokenizer=tokenizer, model=model):
    # prepare inputs and model
    inputs = tokenizer(word, return_tensors="pt") # return pytorch tensors
    outputs = model(**inputs)

    # get embedding from final layer
    last_hidden_states = outputs.last_hidden_state

    # collapse to 1d
    embeddings = torch.squeeze(last_hidden_states, dim=0)

    # convert to list
    return [embed.tolist() for embed in embeddings][1]

In [10]:
for w in ["dog", "puppy", "onomatopoeia", "birthday"]:
    print(np.sqrt(sum([x ** 2 for x in squeeze(w)[1]])))

102.56607688070478
120.19907303624957
69.29362217227322
90.45752579471332


In [13]:
len(squeeze("man")[0])

2048

In [15]:
x = np.array(squeeze("king")[1]) - np.array(squeeze("man")[1])
y = x + np.array(squeeze("woman")[1])
cosine_similarity(y.reshape(1, -1), np.array(squeeze("queen")[1]).reshape(1, -1))

array([[0.24360127]])